<a href="https://colab.research.google.com/github/SprihaT/AND-Logic-Gate/blob/main/BinaryAND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

# Slope function (Derivative function)
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])

    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x + delta_x)

        x[idx] = tmp_val - delta_x
        fx2 = f(x) # f(x - delta_x)
        grad[idx] = (fx1 - fx2) / (2 * delta_x)

        x[idx] = tmp_val # Restore value
        it.iternext()

    return grad

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class LogiGate:
    def __init__(self, gate_name, xdata, tdata):
        self.name = gate_name

        # Creating Input training data
        self.__xdata = xdata.reshape(4, 2) # 4 samples of 2 inputs
        self.__tdata = tdata.reshape(4, 1) # 4 samples of 1 output

        # Hidden layer unit
        self.__W2 = np.random.rand(2, 6) # Weight W2, 2 x 6 matrix
        self.__b2 = np.random.rand(6)

        # Creating output layer unit: Weight W3, Bias b3
        self.__W3 = np.random.rand(6, 1)
        self.__b3 = np.random.rand(1)

        # Learning rate
        self.__learning_rate = 1e-2

        print(self.name + " object is created")

    def feed_forward(self):
        small_number = 1e-7 # No zero in log function

        z2 = np.dot(self.__xdata, self.__W2) + self.__b2 # Output of Hidden Layer
        a2 = sigmoid(z2)

        z3 = np.dot(a2, self.__W3) + self.__b3 # Output of Output layer
        y = sigmoid(z3)

        # Cross-entropy
        return -np.sum(self.__tdata * np.log(y + small_number) +
                       (1 - self.__tdata) * np.log(1 - y + small_number))

    def loss_val(self):
        delta = 1e-7 # No zero in log function

        z2 = np.dot(self.__xdata, self.__W2) + self.__b2 # Output of Hidden Layer
        a2 = sigmoid(z2)

        z3 = np.dot(a2, self.__W3) + self.__b3 # Output of Output Layer
        y = sigmoid(z3)

        # Cross-entropy
        return -np.sum(self.__tdata * np.log(y + delta) +
                       (1 - self.__tdata) * np.log(1 - y + delta))

    # Minimize Cost function
    def train(self):
        f = lambda x: self.feed_forward()

        print("Initial loss value =", self.loss_val())

        for step in range(10001):
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)

            if step % 400 == 0:
                print("step =", step, "loss value =", self.loss_val())

    # Test the output
    def predict(self, xdata):
        z2 = np.dot(xdata, self.__W2) + self.__b2 # Output of Hidden Layer
        a2 = sigmoid(z2)

        z3 = np.dot(a2, self.__W3) + self.__b3 # Output of Output Layer
        y = sigmoid(z3)

        if y > 0.5:
            result = 1 # True
        else:
            result = 0 # False

        return result


# AND() Function and its training
xdata = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
tdata = np.array([[0], [0], [0], [1]])

and_obj = LogiGate("AND", xdata, tdata)
and_obj.train()

AND object is created
Initial loss value = 7.036406762499192
step = 0 loss value = 6.761204116052822
step = 400 loss value = 2.2332600647091683
step = 800 loss value = 2.1069277067961707
step = 1200 loss value = 1.9163263030417301
step = 1600 loss value = 1.631866366297708
step = 2000 loss value = 1.28648359394347
step = 2400 loss value = 0.9664074599012141
step = 2800 loss value = 0.7137850126470175
step = 3200 loss value = 0.5309422207916709
step = 3600 loss value = 0.4031488760882035
step = 4000 loss value = 0.31408382733364537
step = 4400 loss value = 0.2511165301297711
step = 4800 loss value = 0.20559989766264608
step = 5200 loss value = 0.171878979086315
step = 5600 loss value = 0.14628901056881127
step = 6000 loss value = 0.12643272739978742
step = 6400 loss value = 0.1107137227681774
step = 6800 loss value = 0.09804616936396744
step = 7200 loss value = 0.08767525227205356
step = 7600 loss value = 0.07906506224514476
step = 8000 loss value = 0.07182747699992498
step = 8400 loss 

In [4]:
import plotly.graph_objects as go
import numpy as np

# Assuming and_obj is your trained AND gate object

# Create a meshgrid of x and y values
x = np.arange(0, 1.1, 0.1)
y = np.arange(0, 1.1, 0.1)
X, Y = np.meshgrid(x, y)

# Calculate the output z for each combination of x and y
Z = np.zeros_like(X)
for i in range(X.shape[0]):
  for j in range(X.shape[1]):
    input_data = np.array([[X[i, j], Y[i, j]]])
    z = and_obj.predict(input_data)
    Z[i, j] = z


# Create the 3D surface plot
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

fig.update_layout(title='Trained AND Logic Gate',
                  scene=dict(
                      xaxis_title='x',
                      yaxis_title='y',
                      zaxis_title='z (Output)'))

fig.show()
